# Simulation Example

Here we demonstrate how to use this package to estimate the multivariate PSD of VAR(2) simulated data.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sgvb_psd.utils.sim_varma import SimVARMA

np.random.seed(0)

var2 = SimVARMA(
    n_samples=1024,
    var_coeffs=np.array([[[0.5, 0.], [0., -0.3]], [[0., 0.], [0., -0.5]]]),
    vma_coeffs=np.array([[[1., 0.], [0., 1.]]]),
    sigma=np.array([[1., 0.9], [0.9, 1.]])
)
var2

0.50,0.00
0.00,-0.30
0.00,0.00
0.00,-0.50
1.00,0.00
0.00,1.00
1.00,0.90
0.90,1.00


In [2]:
var2.plot()
plt.savefig(f'var2_data.png')
plt.close()

![VAR(2) Data](var2_data.png)

In [3]:
from sgvb_psd.optimal_psd_estimator import OptimalPSDEstimator

optim = OptimalPSDEstimator(
    N_theta=30, nchunks=1, duration=1,
    ntrain_map=100, x=var2.data, max_hyperparm_eval=1
)
optim.run()
_ = optim.plot(
    true_psd=[var2.psd, var2.freq], 
    off_symlog=False,
    xlims=[0, np.pi],
);
plt.savefig(f'var2_psd.png');
plt.close()

2024-09-16 17:46:40.695470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


17:46:45|SGVB-PSD|INFO| 0s |No GPU detected. Running on CPU.
17:46:45|SGVB-PSD|INFO| 0s |Running hyperopt to find optimal learning rate
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]17:46:45|SGVB-PSD|DEBUG| 0s |Inputted data shape: (1024, 2)
17:46:45|SGVB-PSD|DEBUG| 0s |Starting Model Inference Training..
17:46:45|SGVB-PSD|DEBUG| 0s |Start Point Estimating... 
17:46:48|SGVB-PSD|DEBUG| 3s |MAP Training Time: 3.11s
17:46:48|SGVB-PSD|DEBUG| 3s |Start UQ training... 
17:46:57|SGVB-PSD|DEBUG| 12s |VI Time: 9.16s
17:46:57|SGVB-PSD|DEBUG| 12s |Total Inference Training Time: 12.34s
100%|██████████| 1/1 [00:12<00:00, 12.63s/trial, best loss: 904.1318359375]
17:46:58|SGVB-PSD|INFO| 12s |Optimal learning rate found in 12.65s
17:46:58|SGVB-PSD|INFO| 12s |Computing optimal PSD estimation
17:46:59|SGVB-PSD|INFO| 13s |Optimal PSD estimation complete in 1.11s


![VAR(2) PSD](var2_psd.png)